In [5]:
import requests
import pandas as pd
import ast
import os
import time
import numpy as np
from openpyxl.styles import *
from openpyxl.formatting.rule import *
from openpyxl.utils.dataframe import *
from openpyxl.utils import get_column_letter
from openpyxl import *
from datetime import *

In [17]:

# lấy API ẩn từ trang web fireant.vn
stock = 'VNM'
auth_token = 'Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkdYdExONzViZlZQakdvNERWdjV4QkRITHpnSSIsImtpZCI6IkdYdExONzViZlZQakdvNERWdjV4QkRITHpnSSJ9.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmZpcmVhbnQudm4iLCJhdWQiOiJodHRwczovL2FjY291bnRzLmZpcmVhbnQudm4vcmVzb3VyY2VzIiwiZXhwIjoxODg5NjIyNTMwLCJuYmYiOjE1ODk2MjI1MzAsImNsaWVudF9pZCI6ImZpcmVhbnQudHJhZGVzdGF0aW9uIiwic2NvcGUiOlsiYWNhZGVteS1yZWFkIiwiYWNhZGVteS13cml0ZSIsImFjY291bnRzLXJlYWQiLCJhY2NvdW50cy13cml0ZSIsImJsb2ctcmVhZCIsImNvbXBhbmllcy1yZWFkIiwiZmluYW5jZS1yZWFkIiwiaW5kaXZpZHVhbHMtcmVhZCIsImludmVzdG9wZWRpYS1yZWFkIiwib3JkZXJzLXJlYWQiLCJvcmRlcnMtd3JpdGUiLCJwb3N0cy1yZWFkIiwicG9zdHMtd3JpdGUiLCJzZWFyY2giLCJzeW1ib2xzLXJlYWQiLCJ1c2VyLWRhdGEtcmVhZCIsInVzZXItZGF0YS13cml0ZSIsInVzZXJzLXJlYWQiXSwianRpIjoiMjYxYTZhYWQ2MTQ5Njk1ZmJiYzcwODM5MjM0Njc1NWQifQ.dA5-HVzWv-BRfEiAd24uNBiBxASO-PAyWeWESovZm_hj4aXMAZA1-bWNZeXt88dqogo18AwpDQ-h6gefLPdZSFrG5umC1dVWaeYvUnGm62g4XS29fj6p01dhKNNqrsu5KrhnhdnKYVv9VdmbmqDfWR8wDgglk5cJFqalzq6dJWJInFQEPmUs9BW_Zs8tQDn-i5r4tYq2U8vCdqptXoM7YgPllXaPVDeccC9QNu2Xlp9WUvoROzoQXg25lFub1IYkTrM66gJ6t9fJRZToewCt495WNEOQFa_rwLCZ1QwzvL0iYkONHS_jZ0BOhBCdW9dWSawD6iF1SIQaFROvMDH1rg'
url = f'https://restv2.fireant.vn/symbols/{stock}/full-financial-reports?type=4&year=2025&quarter=0&limit=6'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36'


params = {
    'type': '4',
    'year': '2025',
    'quarter': '0',
    'limit': '6'
}

headers = {
    'User-Agent': user_agent,
    'Authorization': auth_token
}

response = requests.get(url, headers=headers, params=params)
response = response.json()  # đổi dữ liệu về dạng json để python đọc được (dạng dict hoặc list)

# chuyển dữ liệu về dạng dataframe
data = pd.DataFrame(response)
# tạo thời gian để cho vào tên file
day = datetime.now().strftime("%d-%m-%y")
# đặt tên cho file
cf_file_raw = f'data_raw/cash_flow/{stock}_cash_flow_{day}.xlsx'
# tạo thư mục riêng cho cash flow statement
os.makedirs('data_raw/cash_flow', exist_ok=True)
data.to_excel(cf_file_raw, index=False)  # lưu file raw vào thư mục riêng

In [34]:
#đọc dữ liệu từ cf_file_raw
df_cf = pd.read_excel(cf_file_raw)
# chuyển cột 'values' từ string về danh sách
def convert_if_string(x):
    if isinstance(x,str):
        return ast.literal_eval(x)
    else:
        return x
# Áp dụng hàm cho từng giá trị trong cột 'values'
df_cf['values'] = df_cf['values'].apply(convert_if_string)
# nổ cột 'values' thành nhiều hàng
df_cf = df_cf.explode('values').reset_index(drop=True)
# chuyển cột 'values' từ danh sách thành nhiều cột
df_normalized = pd.json_normalize(df_cf['values'])
#nối cột df_cf ban đầu với df_normalized
df_cf = pd.concat([df_cf.drop(columns=[
    'values','id','parentID','expanded','level','field']), df_normalized], axis=1)
# giữ lại các cột cần thiết
df_cf = df_cf[['name','year','value']]
df_cf = df_cf.pivot_table(index='name', columns='year', values='value',dropna = False).reset_index()








# tạo thư mục chữa file đã chỉnh sửa
os.makedirs('data_processed/cash_flow', exist_ok=True)
#đặt tên file đã chỉnh sửa
cf_file_processed = f'data_processed/cash_flow/{stock}_cash_flow_{day}_processed.xlsx'
# lưu file đã chỉnh sửa
df_cf.to_excel(cf_file_processed, index=False)

